# Projet Python pour la data science

### Auteurs: Arthur LEROUDIER, Romane LE POTIER

##### L'objectif de ce notebook est de présenter le projet que nous avons effectué dans le cadre de l'unité d'enseignement Python pour la data science à l'ENSAE. Ce projet contient une première partie sur la récupération d'une base de données par web scraping puis une seconde partie nettoyage de la base de données et visualisation et une dernière partie sur la modélisation.

# Quelle est l'importance d'un joueur sur l'issu d'un match de basket ?

##### Pour répondre à cette problématique, nous allons étudier les statistiques de joueurs sur 10 matchs de basket, puis prendre la moyenne des statistiques d'un joueur sur la saison et modéliser la probabilité de gagner le match quand ce joueur est sur le terrain. Dans notre cas, on a choisi Victor Wembanyama qui est un joueur français qui fait sa 1ère saison en NBA.

## Installation préalable

In [4]:
!pip install -q lxml
!pip install webdriver-manager

## Importation des modules utiles

In [5]:
import bs4
import lxml
import pandas as pd
from urllib import request
import os
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

# Récupération et traitement des données

## Extraction des données de 10 matchs de basket par web scraping

##### On a décidé d'extraire les tableaux de données sur le site basketball reference des 10 matchs qui ont eu lieu le 25 octobre 2023.

In [6]:
# à changer pour chaque match avec 1 qui veut dire match gagné et 0 match perdu
url_match="http://bkref.com/pi/shareit/WumlT"
titre = 'Orlando'
result = 1

In [8]:
request_text = request.urlopen(url_match).read()
#print(request_text[:1000])
page = bs4.BeautifulSoup(request_text, "lxml")
#print(page)

tableau_match = page.find('table', {'class' : 'sortable stats_table now_sortable modifying'})
#print(tableau_match)

table_body = tableau_match.find('tbody')
rows = table_body.find_all('tr')
#print(rows[1])

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    #print(cols)

dico_participants = dict()
for row in rows:
    nom = row.find_all('th')
    nom = [ele.text.strip() for ele in nom]
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    cols = nom + cols
    if len(cols) > 0 : 
        dico_participants[cols[0]] = cols[1:]
#dico_participants

table_head = tableau_match.find('thead')
titres_col = table_head.find_all('th')
titres_col = [ele.text.strip() for ele in titres_col[3:]]
#print(titres_col)

data_participants = pd.DataFrame.from_dict(dico_participants,orient='index', columns = titres_col)
data_participants['Résultat'] = [result for index in range(data_participants.shape[0])]
data_participants.to_csv('data/'+titre+'_Stats.csv')
print(data_participants)

                       MP FG FGA    FG% 3P 3PA   3P% FT FTA    FT%  ... DRB  \
Paolo Banchero      27:54  3   6   .500  0   1  .000  6   7   .857  ...   5   
Franz Wagner        27:03  5  12   .417  3   7  .429  6   6  1.000  ...   1   
Wendell Carter Jr.  26:55  4  10   .400  0   2  .000  0   0         ...   4   
Markelle Fultz      24:31  5   9   .556  0   0        0   0         ...   1   
Jalen Suggs         20:51  3  12   .250  2   7  .286  0   0         ...   3   
Cole Anthony        24:05  8  12   .667  1   2  .500  3   3  1.000  ...   7   
Joe Ingles          18:14  0   3   .000  0   3  .000  0   0         ...   4   
Gary Harris         16:57  5   8   .625  1   4  .250  0   0         ...   2   
Moritz Wagner       15:21  0   1   .000  0   1  .000  2   2  1.000  ...   4   
Jonathan Isaac      14:09  4   6   .667  1   3  .333  2   3   .667  ...   3   
Goga Bitadze         4:48  2   2  1.000  0   0        3   4   .750  ...   2   
Anthony Black        4:48  2   2  1.000  0   0      

##### On a donc effectué ce code 20 fois pour les 20 équipes différentes en rajoutant aux données la dernière colonne résultat. Ces 20 tableaux se trouvent dans le dossier data.

##### Pour chaque tableau, on a à gauche le nom des joueurs et pour les colonnes:
##### MP: temps joué dans le match
##### FG: tirs marqués à 2 points
##### FGA: tirs tentés à 2 points
##### FG%: pourcentage de réussite à 2 points
##### 3P: tirs marqués à 3 points
##### 3PA: tirs tentés à 3 points
##### 3P%: pourcentage de réussite à 3 points
##### FT: lancers francs marqués
##### FTA: lancers francs tentés
##### FT%: pourcentage de lancers francs réussis
##### ORB: nombre de rebonds offensifs
##### DRB: nombre de rebonds défensifs
##### TRB: nombre de rebonds totaux
##### AST: nombre de passs décisives
##### STL: nombre d'interceptions
##### BLK: nombre de contres
##### TOV: nombre de balles perdues
##### PF: nombre de fautes commises
##### PTS: nombre de points marqués
##### +/-: différence de points marqués entre les deux équipes quand le joueur  est sur le terrain
##### Résultat: 1 si match gagné, 0 sinon

# Nettoyage de la base de données

In [10]:
#commande à n'exécuter qu'une fois
os.chdir('data/')

### On commence par concaténer nos 20 tableaux en un seul tableau.

In [11]:
#on prend la liste des fichiers
files = [f for f in os.listdir('.') if os.path.isfile(f)]
data = pd.read_csv(files[0])
#la colonne +/- n'est pas prise en compte car on a pas sa moyenne pour les matchs de Wembanyama
data = data.loc[:, data.columns!='+/-']
#conversion des minutes par match en flottant
data.loc[:,'MP'] = [float(x.split(':')[0])+ 0.006*float(x.split(':')[1]) for x in data.loc[:,'MP']]
#concatenation de toutes les tables csv en un seul tableau
for file in files[1:]:
    match = pd.read_csv(file)
    match = match.loc[:,match.columns!='+/-']
    match.loc[:,'MP'] = [float(x.split(':')[0])+ 0.006*float(x.split(':')[1]) for x in match.loc[:,'MP']]
    data = pd.concat([data,match])
data.reset_index(drop=True, inplace=True)

print("nombre d'individus: ", len(data.index))

nombre d'individus:  223


##### On a donc les données d'un match pour 223 joueurs différents.

##### Pour certains joueurs, des informations manquent et des cases du tableau sont donc vides. On remplace ces cases vides par une donnée de la même colonne pour des joueurs ayant la même valeur dans la colonne résultat.

In [12]:
#imputation des données manquantes
#remplace les données absentes par une donnée de la même colonne, avec la même valeur pour la variable Résultat
data_win = data[data['Résultat'] == 1]
data_lose = data[data['Résultat'] != 1]
for col in data_win.columns:
    for i,rows in data_win.iterrows():
        if pd.isna(data_win.loc[i,col]):
            data_win.loc[i,col] = np.random.choice(data_win[data_win[col].notna()].loc[:,col])
for col in data_lose.columns:
    for i,rows in data_lose.iterrows():
        if pd.isna(data_lose.loc[i,col]):
            data_lose.loc[i,col] = np.random.choice(data_lose[data_lose[col].notna()].loc[:,col])
data = pd.concat([data_win,data_lose])

X_train = data.loc[:, data.columns!='Résultat']
X_train = X_train.iloc[:,1:]
Y = data.loc[:,'Résultat']
VW = np.array([30.2 ,6.9 ,15.9 ,0.435 ,1.4 ,4.9 ,0.278 ,3.3 ,4.2 ,0.774 ,2.3 ,8.4 ,10.7 ,2.8 ,1.4 ,3.0 ,3.3 ,2.5 ,18.5])